# Musical Graph

Author: Pablo Villanueva-Domingo

In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
datapath = "data/"

# Select data for each musical scene
#dataname = "neofolk"
dataname = "norwegian_black_metal"
#dataname = "valencian"
dataname = "jazz"
#dataname = "extremoduro"
#dataname = "rock_radical_vasco"
#dataname = "folk_valencia"

filename = "data/"+dataname+".txt"

# Flag for showing collaborations
show_colls = True

dataframe = pd.read_csv(filename, sep=';', names=['Band', 'Members', 'Collaborations'])
dataframe = dataframe.where(pd.notna(dataframe), None)
dataframe.head()

,Band,Members,Collaborations
0,Dizzy Gillespie,"Cecil Payne,Miles Davis,Stan Getz,Max Roach",NaN
1,Charlie Parker,"Max Roach,Miles Davis",NaN
2,Bud Powerll,"Max Roach,Paul Chambers",NaN
3,Thelonious Monk,"Coleman Hawkins,John Coltrane,Art Blakey,Max R...",NaN
4,Charles Mingus,Max Roach,NaN


In [3]:
def create_graph(dataframe, coll=True):

    # Create an empty graph
    graph = nx.Graph()
    
    # First, populate the graph nodes with all the considered bands 
    for i, band in dataframe.iterrows():

        graph.add_node(band["Band"], type="band")

    # Second, add members
    for i, band in dataframe.iterrows():
        
        if band["Members"] is not None:

            for member in band["Members"].split(","):
                if not (member in graph):

                    graph.add_node(member, type="person")

                graph.add_edge(band["Band"], member, type="member")
            
        # Then, add collaborations
        # If there are elements in the collaborations column
        if not dataframe['Collaborations'].isnull().values.sum()==len(dataframe):
            if coll and band["Collaborations"] is not None:
                for person in band["Collaborations"].split(","):
                    if not (person in graph):

                        graph.add_node(person, type="person")

                    if not graph.has_edge(band["Band"], person):
                        graph.add_edge(band["Band"], person, type="collab")
            
        
    return graph

graph = create_graph(dataframe, coll=show_colls)

In [4]:
from ipysigma import Sigma

args = {"node_size":graph.degree,
        "node_color":"type",
        "edge_color":"type",
        "label_density":10,
        "node_border_color_from":"node",
        "start_layout":2}

sig = Sigma(graph, **args)
sig

Sigma(nx.Graph with 46 nodes and 89 edges)

In [5]:
sig.write_html(graph,"htmls/"+dataname+'.html', fullscreen=True, **args)